In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import config
from scipy.stats.stats import ttest_ind

%matplotlib inline
from sqlalchemy import create_engine
import warnings

warnings.filterwarnings("ignore")

<IPython.core.display.Javascript object>

In [16]:
postgres_user = config.user
postgres_pw = config.password
postgres_host = config.host
postgres_port = config.port
postgres_db = "useducation"

engine = create_engine(
    "postgresql://{}:{}@{}:{}/{}".format(
        postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db
    )
)

df = pd.read_sql_query("select * from useducation", con=engine)

# No need for an open connection,
# because you're only doing a single query
engine.dispose()

<IPython.core.display.Javascript object>

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1492 entries, 0 to 1491
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   PRIMARY_KEY                   1492 non-null   object 
 1   STATE                         1492 non-null   object 
 2   YEAR                          1492 non-null   int64  
 3   ENROLL                        1229 non-null   float64
 4   TOTAL_REVENUE                 1280 non-null   float64
 5   FEDERAL_REVENUE               1280 non-null   float64
 6   STATE_REVENUE                 1280 non-null   float64
 7   LOCAL_REVENUE                 1280 non-null   float64
 8   TOTAL_EXPENDITURE             1280 non-null   float64
 9   INSTRUCTION_EXPENDITURE       1280 non-null   float64
 10  SUPPORT_SERVICES_EXPENDITURE  1280 non-null   float64
 11  OTHER_EXPENDITURE             1229 non-null   float64
 12  CAPITAL_OUTLAY_EXPENDITURE    1280 non-null   float64
 13  GRA

<IPython.core.display.Javascript object>

In [22]:
df["GRADES_4_G"].sum() - df["GRADES_8_G"].sum()

364571.0

<IPython.core.display.Javascript object>

In [21]:
df["GRADES_8_G"].sum()

87472909.0

<IPython.core.display.Javascript object>

In [4]:
df.head()

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
0,1992_ALABAMA,ALABAMA,1992,NaN,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,...,57948.0,58025.0,41167.0,471564.0,196386.0,676174.0,208.327876,252.187522,207.963517,NaN
1,1992_ALASKA,ALASKA,1992,NaN,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,...,9748.0,8789.0,6714.0,79117.0,30847.0,112335.0,NaN,NaN,NaN,258.859712
2,1992_ARIZONA,ARIZONA,1992,NaN,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0,...,55433.0,49081.0,37410.0,437127.0,175210.0,614881.0,215.253932,265.366278,206.212716,262.169895
3,1992_ARKANSAS,ARKANSAS,1992,NaN,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0,...,34632.0,36011.0,27651.0,281338.0,123113.0,405259.0,210.206028,256.312090,208.634458,264.619665
4,1992_CALIFORNIA,CALIFORNIA,1992,NaN,26260025.0,2072470.0,16546514.0,7641041.0,27138832.0,14358922.0,...,418418.0,363296.0,270675.0,3286034.0,1372011.0,4717112.0,208.398961,260.892247,196.764414,NaN


<IPython.core.display.Javascript object>

Say you want to understand the relationship between government expenditures and the students' overall success in math and reading.

1. Compute the average score for each row for all students weighted properly by grade. In other words, each row will have its own average. Notice that the number of students in the fourth grade isn't the same as the number of students in the eighth grade. So, you should appropriately weigh the scores.
2. What are the correlations between this newly created score variable and the expenditure types? Which one of the expenditure types is more correlated than the others?
3. Now, apply PCA to the four expenditure types. How much of the total variance is explained by the first component?
4. What is the correlation between the overall score variable and the first principal component?
5. If you were to choose the best variables for your model, would you prefer using the first principal component instead of the expenditure variables? Why?

## 1. Compute the average score for each row for all students weighted properly by grade. In other words, each row will have its own average. Notice that the number of students in the fourth grade isn't the same as the number of students in the eighth grade. So, you should appropriately weigh the scores.

In [6]:
fill_list = ["ENROLL", "TOTAL_REVENUE", "FEDERAL_REVENUE",
             "STATE_REVENUE", "LOCAL_REVENUE", "TOTAL_EXPENDITURE",
             "INSTRUCTION_EXPENDITURE", "SUPPORT_SERVICES_EXPENDITURE",
             "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE", "GRADES_PK_G",
             "GRADES_KG_G", "GRADES_4_G", "GRADES_8_G", "GRADES_12_G", "GRADES_1_8_G",
             "GRADES_9_12_G", "GRADES_ALL_G"]

states = df['STATE'].unique()

for state in states:
    df.loc[df['STATE'] == state, fill_list] = df.loc[df['STATE'] == state, fill_list].interpolate()

# drop the remaining NaN's after interplotion.    
df.dropna(inplace = True)


<IPython.core.display.Javascript object>

In [7]:
df["overall score"] = (
    df["GRADES_4_G"] * (df["AVG_MATH_4_SCORE"] + df["AVG_READING_4_SCORE"] * 0.5)
    + df["GRADES_8_G"] * (df["AVG_MATH_8_SCORE"] + df["AVG_READING_8_SCORE"] * 0.5)
) / (df["GRADES_4_G"] + df["GRADES_8_G"])

<IPython.core.display.Javascript object>

The score variable is weighted using the number of students in the respective grades.

## 2. What are the correlations between this newly created score variable and the expenditure types? Which one of the expenditure type is more correlated than the others?

In [9]:
df.columns

Index(['PRIMARY_KEY', 'STATE', 'YEAR', 'ENROLL', 'TOTAL_REVENUE',
       'FEDERAL_REVENUE', 'STATE_REVENUE', 'LOCAL_REVENUE',
       'TOTAL_EXPENDITURE', 'INSTRUCTION_EXPENDITURE',
       'SUPPORT_SERVICES_EXPENDITURE', 'OTHER_EXPENDITURE',
       'CAPITAL_OUTLAY_EXPENDITURE', 'GRADES_PK_G', 'GRADES_KG_G',
       'GRADES_4_G', 'GRADES_8_G', 'GRADES_12_G', 'GRADES_1_8_G',
       'GRADES_9_12_G', 'GRADES_ALL_G', 'AVG_MATH_4_SCORE', 'AVG_MATH_8_SCORE',
       'AVG_READING_4_SCORE', 'AVG_READING_8_SCORE', 'overall score',
       'overall_score'],
      dtype='object')

<IPython.core.display.Javascript object>

In [10]:
df[['overall score', 'TOTAL_EXPENDITURE', 'INSTRUCTION_EXPENDITURE', 'SUPPORT_SERVICES_EXPENDITURE', 'OTHER_EXPENDITURE', 'CAPITAL_OUTLAY_EXPENDITURE']].corr()


,overall score,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
overall score,1.000000,0.078683,0.089648,0.083896,0.016043,0.003239
TOTAL_EXPENDITURE,0.078683,1.000000,0.990255,0.992988,0.943702,0.925784
INSTRUCTION_EXPENDITURE,0.089648,0.990255,1.000000,0.974399,0.902173,0.884798
SUPPORT_SERVICES_EXPENDITURE,0.083896,0.992988,0.974399,1.000000,0.952282,0.911245
OTHER_EXPENDITURE,0.016043,0.943702,0.902173,0.952282,1.000000,0.918188
CAPITAL_OUTLAY_EXPENDITURE,0.003239,0.925784,0.884798,0.911245,0.918188,1.000000


<IPython.core.display.Javascript object>

* The 'INSTRUCTION_EXPENDITURE' is the most highly correlated with the 'overall score'.

## 3. Now, apply PCA to the four expenditure types. How much of the total variance is explained by the first component?

In [11]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

<IPython.core.display.Javascript object>

In [14]:
X = df[
    [
        "INSTRUCTION_EXPENDITURE",
        "SUPPORT_SERVICES_EXPENDITURE",
        "OTHER_EXPENDITURE",
        "CAPITAL_OUTLAY_EXPENDITURE",
    ]
]

X = StandardScaler().fit_transform(X)

sklearn_pca = PCA(n_components=1)
df["pca_1"] = sklearn_pca.fit_transform(X)

print(
    "The percentage of total variance in the dataset explained by each",
    "component from SKlearn PCA. \n",
    sklearn_pca.explained_variance_ratio_,
)

The percentage of total variance in the dataset explained by each component from SKlearn PCA. 
 [0.9430175]


<IPython.core.display.Javascript object>

About 94% of the total variance is explained by the first principal component.

## 4. What is the correlation between the overall score variable and the first principal component?

In [15]:
df[
    [
        "overall_score",
        "pca_1",
        "TOTAL_EXPENDITURE",
        "INSTRUCTION_EXPENDITURE",
        "SUPPORT_SERVICES_EXPENDITURE",
        "OTHER_EXPENDITURE",
        "CAPITAL_OUTLAY_EXPENDITURE",
    ]
].corr()

,overall_score,pca_1,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
overall_score,1.000000,0.044596,0.074155,0.087386,0.079002,0.010959,-0.005389
pca_1,0.044596,1.000000,0.992054,0.968606,0.988345,0.971343,0.955792
TOTAL_EXPENDITURE,0.074155,0.992054,1.000000,0.990255,0.992988,0.943702,0.925784
INSTRUCTION_EXPENDITURE,0.087386,0.968606,0.990255,1.000000,0.974399,0.902173,0.884798
SUPPORT_SERVICES_EXPENDITURE,0.079002,0.988345,0.992988,0.974399,1.000000,0.952282,0.911245
OTHER_EXPENDITURE,0.010959,0.971343,0.943702,0.902173,0.952282,1.000000,0.918188
CAPITAL_OUTLAY_EXPENDITURE,-0.005389,0.955792,0.925784,0.884798,0.911245,0.918188,1.000000


<IPython.core.display.Javascript object>

The correlation of the first principal component is less than the instruction expenditure and the support service expenditure variables.

## 5. If you were to choose the best variables for your model, would you prefer using the first principal component instead of the expenditure variables? Why?

The instruction expenditure variable is more correlated with the overall score than the first principal component. Hence using instruction expenditure makes more sense. PCA works best when the correlations between the variables are less then and equal to 0.8. In our case, all of the expenditure variable are highly correlated with each other. This may result in instable principal component estimations.